In [20]:
import paho.mqtt.client as mqtt

client_publish = mqtt.Client("Python2")
client_publish.connect(host="test.mosquitto.org", port=1883)
client_publish.publish("IC.embedded/Iceberg", "Please work dear lord!")
client_publish.disconnect()


0

In [1]:
import smbus
import time
import math

bus = smbus.SMBus(1)
i2c_addr_flex = 0x48 # our flex sensor slave address
i2c_addr_compass = 0x0c # our compass sensor slave address

In [2]:
def read_flex (i2c_addr, duration, no_of_reading):
    #below are the 4 different kind of register in ADC1115 module
    status_reg = 0x00
    mode_reg = 0x01
    dataH_reg = 0x02
    dataL_reg = 0x03
    
    #this is to configure the device
    bus.write_word_data(i2c_addr, mode_reg, 0x8384)

    #this is to write to pointer register
    bus.write_byte(i2c_addr, 0)

    while(no_of_reading):
        val_swapped = bus.read_word_data(i2c_addr, 0x00)
        val = (val_swapped & 0xFF) <<8 | (val_swapped >>8)
        print ("flex reading: ", val)
        time.sleep(duration/no_of_reading)
        no_of_reading = no_of_reading - 1

In [3]:
def read_compass(i2c_addr, duration, no_of_reading):
    
    while(no_of_reading):
        # Select write register command, 0x60(96)
        # AH = 0x00, AL = 0x5C, GAIN_SEL = 5, Address register (0x00 << 2)
        config = [0x00, 0x5C, 0x00]
        bus.write_i2c_block_data(i2c_addr, 0x60, config)

        # Read data back, 1 byte, (ACK byte)
        # Status byte
        data = bus.read_byte(i2c_addr)

        # AH = 0x02, AL = 0xB4, RES for magnetic measurement = 0, Address register (0x02 << 2)
        config = [0x02, 0xB4, 0x08]
        bus.write_i2c_block_data(i2c_addr, 0x60, config)

        # Read data back, 1 bytem (ACK byte)
        # Status byte
        data = bus.read_byte(i2c_addr)

        # Start single meaurement mode, X, Y, Z-Axis enabled
        bus.write_byte(i2c_addr, 0x3E)

        # Read data back, 1 byte
        # Status byte
        data = bus.read_byte(0x0C)

        time.sleep(0.5)

        # Read data back from 0x4E(78), 7 bytes
        # Status, xMag msb, xMag lsb, yMag msb, yMag lsb, zMag msb, zMag lsb
        data = bus.read_i2c_block_data(i2c_addr, 0x4E, 7)
        print(data)

        xMag = data[1] * 256 + data[2]
        if xMag > 32767 :
            xMag -= 65536

        yMag = data[3] * 256 + data[4]
        if yMag > 32767 :
            yMag -= 65536

        zMag = data[5] * 256 + data[6]
        if zMag > 32767 :
            zMag -= 65536
            
        heading = math.atan2(yMag, xMag) * 180 / math.pi
        

        # Output data to screen
        print("Magnetice Field (x, y, z) = ", xMag, yMag, zMag)
        print(heading + 330)
        time.sleep(duration/no_of_reading)
        no_of_reading = no_of_reading - 1

        
        #if magnetic field change too much then nearby must have ferous object
        
    

In [14]:
read_flex(i2c_addr_flex, 2, 2)
read_compass(i2c_addr_compass, 2, 2)

flex reading:  16906
flex reading:  16911
Magnetice Field (x, y, z) =  39 -33 72
-40.23635830927382
Magnetice Field (x, y, z) =  32 -29 69
-42.18444331578877


In [21]:
read_compass(i2c_addr_compass, 10, 5)

[2, 0, 22, 255, 207, 0, 64]
Magnetice Field (x, y, z) =  22 -49 64
-65.82089285331082
[2, 0, 28, 255, 207, 0, 64]
Magnetice Field (x, y, z) =  28 -49 64
-60.25511870305779
[2, 0, 26, 255, 212, 0, 65]
Magnetice Field (x, y, z) =  26 -44 65
-59.42077312751099
[2, 0, 30, 255, 208, 0, 62]
Magnetice Field (x, y, z) =  30 -48 62
-57.9946167919165
[2, 0, 27, 255, 209, 0, 63]
Magnetice Field (x, y, z) =  27 -47 63
-60.124007308310574


In [40]:
import time
import paho.mqtt.client as mqtt
print("Done")
client = mqtt.Client("")
print("Here1")
client.connect("test.mosquitto.org", port=1883)
print("Here2")
client.publish("IC.embedded/Faraday", "HEY MAN")
print("Here3")
print("done")

Done
Here1
Here2
Here3
done


In [11]:
read_compass(i2c_addr_compass, 10, 5)

[2, 255, 207, 255, 111, 0, 46]
Magnetice Field (x, y, z) =  -49 -145 46
221.32829310517087
[2, 255, 216, 255, 117, 0, 46]
Magnetice Field (x, y, z) =  -40 -139 46
223.94578033722425
[2, 255, 169, 255, 191, 0, 54]
Magnetice Field (x, y, z) =  -87 -65 54
186.76438067579048
[2, 255, 167, 255, 194, 0, 34]
Magnetice Field (x, y, z) =  -89 -62 34
184.86219614691484
[2, 255, 166, 255, 192, 0, 33]
Magnetice Field (x, y, z) =  -90 -64 33
185.4170552764674


In [12]:
read_compass(i2c_addr_compass, 10, 5)

[2, 255, 218, 255, 111, 0, 50]
Magnetice Field (x, y, z) =  -38 -145 50
225.3148029628785
[2, 255, 212, 255, 111, 0, 53]
Magnetice Field (x, y, z) =  -44 -145 53
223.11966983447616
[2, 255, 250, 255, 118, 0, 81]
Magnetice Field (x, y, z) =  -6 -138 81
237.51044707800085
[2, 0, 10, 255, 156, 0, 99]
Magnetice Field (x, y, z) =  10 -100 99
245.71059313749964
[2, 255, 223, 255, 105, 0, 42]
Magnetice Field (x, y, z) =  -33 -151 42
227.6722292886831
